# Multiple Regression

In questo notebook saranno utilizzati i dati delle vendite delle case in King County per predire i prezzi, utilizzando la multiple regression
Cosa Faremo :

* Utilizzo di funzioni in Graphlab per calcolare i coefficienti della regressione
* Dati i coefficienti, i predittori e i pesi, scrivere una funzione per calcolare la RSS.
* Guardare i coefficienti e comprendere il loro significato.
* Valutare i modelli tramite la RSS

# Import di  graphlab create


In [13]:
import graphlab

# Caricamento dei dati di input
Consideriamo un dataset contentente i prezzi delle case vendute nella contea di King tra Maggio del 2014 e Maggio del 2015

In [14]:
sales = graphlab.SFrame.read_csv('data/kc_house_data.csv')

Finished parsing file /Users/Administrator/Development/GraphLabProjects/IA/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.094295 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/Administrator/Development/GraphLabProjects/IA/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.089518 secs.

# Divido i dati in training set e test set

In [15]:
train_data,test_data = sales.random_split(.8,seed=0)

# Costruisci la multiple linear regression  

Consideriamo un modello di multiple linear regression che considera:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] sui dati di training:



In [16]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', 
                                                  features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.025693     | 4146407.590959     | 258679.804259 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Estraiamo i coefficienti calcolati dal modello

In [17]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0692067  | 7873.33813675 |
| sqft_living |  None | 315.403441118  | 3.45570032294 |
|   bedrooms  |  None | -65080.2154416 | 2717.45685213 |
|  bathrooms  |  None | 6944.02087632  | 3923.11492813 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



In [18]:
example_model.show()

Canvas is accessible via web browser at the URL: http://localhost:49345/index.html
Opening Canvas in default web browser.


# Fare predizioni

Utilizziamo la funzione predict, già pronta in graphlab, per predire i risultati dei dati di training, basandosi sul modello creato

In [19]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.504278


In [26]:
train_data['price'].head()

dtype: float
Rows: 10
[221900.0, 538000.0, 180000.0, 604000.0, 510000.0, 1225000.0, 257500.0, 291850.0, 229500.0, 323000.0]

In [27]:
example_predictions.head()

dtype: float
Rows: 10
[271789.50427757535, 718880.3135271334, 207554.3088607718, 466612.0146607226, 436435.24571294733, 1568323.9522438196, 449210.37137116166, 237413.101781566, 461031.5689484328, 506141.9787859095]

In [31]:
e = (train_data['price'][0]-example_predictions[0])
print e

-49889.5042776


# Calcolo del RSS

Dato il modello, i dati e l'output, calcoliamo RSS

In [32]:
def get_residual_sum_of_squares(model, data, outcome):
    # Calcoliamo le prediction
    predictions = model.predict(data)
    # Calcolo degli errori
    residuals = predictions - outcome
    # calcolo della RSS
    RSS = (residuals * residuals).sum()
    return(RSS)    

Test della funzione sui dati di test

In [11]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Creazione di nuove features

In [7]:
from math import log

E’ possibile considerare come nuove features anche trasformazioni delle feature esistenti con lo scopo di ottenere una migliore predizione.
Creiamo 4 nuove features :
* bedrooms_squared = (bedrooms*bedrooms). 
Questo permetterà di creare una distinzione netta tra le case con poche camere
da letto (es: 1) e quelle con molte camere (es: 4) in quanto 1^2 =1, ma 4^2 =16.

* bed_bath_rooms = bedrooms*bathrooms
E’ anche chiamata “feature interaction” ed è più alta quando ci sono sia tante
camere che tanti bagni

* log_sqft_living = log(sqft_living)
Fare il logaritmo ha l’effetto di portare i valori più grandi vicini e di espandere
quelli più piccoli

* lat_plus_long = lat + long

In [27]:
 train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)
train_data['bed_plus_bath'] = train_data.apply(lambda x: x['bedrooms'] * x['bathrooms'])
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data.apply(lambda x: x['lat'] + x['long'])
test_data['bed_plus_bath'] = test_data.apply(lambda x: x['bedrooms'] * x['bathrooms'])
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
test_data['lat_plus_long'] = test_data.apply(lambda x: x['lat'] + x['long'])

In [38]:
print test_data['bedrooms_squared'].mean()
print test_data['bed_bath_rooms'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

12.4466777016
7.50390163159
7.55027467965
-74.6533349722


# Confronto tra modelli

Definisco 3 nuovi modelli, con le nuove features create.
* Modello 1: sqft_living, #bedrooms, #bathrooms, latitude and longitude
* Modello 2: Modello 1+ bed_bath_rooms
* Modello 3: Modello 2 + bedrooms_squared, log_sqft_living,lat_plus_long.

In [31]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Ora che ho le features, posso imparare i parametri per i tre differenti modelli nel predire il prezzo utilizzando graphlab.linear_regression.create() e guardando il valore dei coefficienti. 

In [33]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 5
PROGRESS: Number of unpacked features : 5
PROGRESS: Number of coefficients    : 6
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.016001     | 4074878.213096     | 236378.596455 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 6
PROGRESS: Numbe

In [36]:
#estrazione dei coefficienti per ciascun modello 
print 'model 1'
print model_1.get("coefficients")
print 'model 2'
print model_2.get("coefficients")
print 'model 3'
print model_3.get("coefficients")

model 1
+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | -56140675.7444 |
| sqft_living |  None | 310.263325778  |
|   bedrooms  |  None | -59577.1160682 |
|  bathrooms  |  None | 13811.8405418  |
|     lat     |  None | 629865.789485  |
|     long    |  None | -214790.285186 |
+-------------+-------+----------------+
[6 rows x 3 columns]

model 2
+----------------+-------+----------------+
|      name      | index |     value      |
+----------------+-------+----------------+
|  (intercept)   |  None | -54410676.1152 |
|  sqft_living   |  None | 304.449298057  |
|    bedrooms    |  None | -116366.043231 |
|   bathrooms    |  None | -77972.3305135 |
|      lat       |  None | 625433.834953  |
|      long      |  None | -203958.60296  |
| bed_bath_rooms |  None | 26961.6249092  |
+----------------+-------+----------------+
[7 rows x 3 columns]

model 3
+------------------+-------+----------

# Quale modello è il migliore? 

In [41]:
#Calcolo la RSS per ognuno dei tre modelli sui dati di TRAINING
print get_residual_sum_of_squares(model_1, train_data, train_data['price'])
print get_residual_sum_of_squares(model_2, train_data, train_data['price'])
print get_residual_sum_of_squares(model_3, train_data, train_data['price'])

9.71328233544e+14
9.61592067856e+14
9.05276314555e+14


In [42]:
#Calcolo la RSS per ognuno dei tre modelli sui dati di TEST
print get_residual_sum_of_squares(model_1, test_data, test_data['price'])
print get_residual_sum_of_squares(model_2, test_data, test_data['price'])
print get_residual_sum_of_squares(model_3, test_data, test_data['price'])

2.26568089093e+14
2.24368799994e+14
2.51829318952e+14


** Il modello migliore è model_2 poichè ha RSS sui test_data minore ** 